# Not for the public Eye

In [1]:
import ants
import numpy as np
import os

In [58]:
_atlas_fiber_DTI_path = "tissue_reconstruction/data/FSL_HCP1065_tensor_1mm_space-HPC-AntsIndexSpace_SRI.nii.gz"
_atlas_t1_path = "tissue_reconstruction/data/sub-mni152_t1-inside-brain_space-sri.nii.gz"
_patient_path = "tissue_reconstruction/data/sample_patient.nii.gz"


tensor_image = ants.image_read(_atlas_fiber_DTI_path)
atlas_t1 = ants.image_read(_atlas_t1_path)
patient = ants.image_read(_patient_path)

reg = ants.registration(patient, atlas_t1, "SyN")

transform = reg['fwdtransforms']

print(tensor_image)
print(atlas_t1)
print(patient)
print(transform[0])

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (239.0, 0.0, 0.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (239.0, 0.0, 0.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

/tmp/tmp7vmg5dwb1Warp.nii.gz


In [3]:
# Optionally for the Fututre combine the affine and the warp ...

def combine_warp(fwd_transform_paths):
    return ants.image_read(fwd_transform_paths[0])

In [ ]:
def func1(fixed_image, dti_img, fwd_transform_paths):
    # Using ANTs registration throught the system
    warp_path = fwd_transform_paths[0]    
    
    # 1. Split into Six components
    dti_components = ants.split_channels(dti_img)
    #print(dti_components)

    # 2. Transform every component
    transformed_components = []
    for component in dti_components:
        transformed_component = ants.apply_transforms(fixed=fixed_image, 
                                                    moving=component, 
                                                    transformlist=fwd_transform_paths)
        transformed_components.append(transformed_component)
    transformed_dti_img = ants.merge_channels(transformed_components)

    print(transformed_dti_img)
    
    # 3. Reorient Image ?!
    transformed_dti_save_path = "out/transformed_dti.nii.gz"
    transformed_rotated_dti_save_path = "out/transformed_rotated_dti.nii.gz"

    ants.image_write(transformed_dti_img, transformed_dti_save_path)

    reorientTensor = f"ReorientTensorImage 3 {transformed_dti_save_path} {transformed_rotated_dti_save_path} {warp_path}"
    os.system(reorientTensor) 

func1(patient, tensor_image, transform)
rotated_tensor1 = ants.image_read("out/transformed_rotated_dti.nii.gz")
transformed_tensor_img = ants.image_read("out/transformed_dti.nii.gz")


ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

FIELD: /tmp/tmpxcf5772f1Warp.nii.gz
moving_image_filename: out/transformed_dti.nii.gz
output_image_filename: out/transformed_rotated_dti.nii.gz
[0/1]: FIELD: /tmp/tmpxcf5772f1Warp.nii.gz
Warp transform
Iterating over image


In [54]:
import ants
import numpy as np
from scipy.ndimage import map_coordinates
from numpy.linalg import svd
import os
from tqdm import tqdm

def get_local_jacobian(deformation_field, point, spacing=(1.0, 1.0, 1.0)):
    """
    Compute the Jacobian matrix of the deformation field at a given point.
    
    Parameters:
    -----------
    deformation_field : numpy.ndarray
        Array of shape [x, y, z, 3] containing the deformation vectors
    point : tuple
        (x, y, z) coordinates where to compute the Jacobian
    spacing : tuple
        Voxel spacing in mm
        
    Returns:
    --------
    numpy.ndarray
        3x3 Jacobian matrix
    """
    x, y, z = point
    jacobian = np.zeros((3, 3))
    
    # Compute partial derivatives using central differences
    for i in range(3):  # For each component of the deformation field
        # x derivative
        if 0 < x < deformation_field.shape[0]-1:
            jacobian[0, i] = (deformation_field[x+1, y, z, i] - 
                            deformation_field[x-1, y, z, i]) / (2 * spacing[0])
        # y derivative
        if 0 < y < deformation_field.shape[1]-1:
            jacobian[1, i] = (deformation_field[x, y+1, z, i] - 
                            deformation_field[x, y-1, z, i]) / (2 * spacing[1])
        # z derivative
        if 0 < z < deformation_field.shape[2]-1:
            jacobian[2, i] = (deformation_field[x, y, z+1, i] - 
                            deformation_field[x, y, z-1, i]) / (2 * spacing[2])
    
    # Add identity matrix since the deformation field represents displacement
    jacobian = np.eye(3) + jacobian
    return jacobian

def get_rotation_from_jacobian(jacobian):
    """
    Extract the rotation matrix from a Jacobian matrix using SVD.
    
    Parameters:
    -----------
    jacobian : numpy.ndarray
        3x3 Jacobian matrix
        
    Returns:
    --------
    numpy.ndarray
        3x3 rotation matrix
    """
    U, _, Vh = svd(jacobian)
    R = np.dot(U, Vh)
    
    # Ensure proper rotation (determinant = 1)
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = np.dot(U, Vh)
    return R

def func2(fixed_image, dti_img, fwd_transform_paths):
    """
    Transform DTI data using a deformation field with proper tensor reorientation.
    
    Parameters:
    -----------
    """
    
    # Load deformation field using ANTs and convert to numpy
    combined_warp = combine_warp(fwd_transform_paths)
    def_field_np = combined_warp.numpy()
    
    # Initialize output array
    dti_data = dti_img.numpy()
    transformed_dti = np.zeros_like(dti_data)
    
    # Convert between 6D vector and 3x3 matrix representations
    def vector_to_matrix(vector):
        return np.array([[vector[0], vector[1], vector[2]],
                        [vector[1], vector[3], vector[4]],
                        [vector[2], vector[4], vector[5]]])
    
    def matrix_to_vector(matrix):
        return np.array([matrix[0,0], matrix[0,1], matrix[0,2],
                        matrix[1,1], matrix[1,2], matrix[2,2]])
    
    # Create coordinate grid
    shape = dti_data.shape[:3]
    grid = np.stack(np.meshgrid(
        np.arange(shape[0]),
        np.arange(shape[1]),
        np.arange(shape[2]),
        indexing='ij'
    ), axis=-1)
    
    # Apply deformation and tensor reorientation
    for x in tqdm(range(shape[0])):
        for y in range(shape[1]):
            for z in range(shape[2]):
                # Get current point
                point = (x, y, z)
                
                # Compute local Jacobian and extract rotation
                jacobian = get_local_jacobian(def_field_np, point, fixed_image.spacing)
                rotation = get_rotation_from_jacobian(jacobian)
                
                # Transform tensor
                tensor_vec = dti_data[x, y, z, :]
                tensor_mat = vector_to_matrix(tensor_vec)
                transformed_tensor = np.matmul(
                    np.matmul(rotation, tensor_mat),
                    rotation.T
                )
                
                # Get transformed coordinates
                new_coords = np.array([x, y, z]) + def_field_np[x, y, z]
                
                # Store transformed tensor if coordinates are within bounds
                if (0 <= new_coords[0] < shape[0] and 
                    0 <= new_coords[1] < shape[1] and 
                    0 <= new_coords[2] < shape[2]):
                    transformed_dti[int(new_coords[0]), 
                                 int(new_coords[1]), 
                                 int(new_coords[2])] = matrix_to_vector(transformed_tensor) 
    
    transformed_dti_img = ants.from_numpy(transformed_dti, fixed_image.origin, fixed_image.spacing, fixed_image.direction, has_components=True)
    return transformed_dti_img

rotated_tensor2 = func2(patient, tensor_image, transform)

  3%|▎         | 8/240 [01:26<41:45, 10.80s/it]


KeyboardInterrupt: 

In [51]:
from scipy.ndimage import sobel

def compute_jacobian_field(deformation_field):
    """
    Compute the Jacobian field using vectorized gradient operations on the displacement field.
    
    Parameters:
    - displacement_field: A 4D numpy array of shape (X, Y, Z, 3).
    
    Returns:
    - jacobian_field: A 5D numpy array of shape (X, Y, Z, 3, 3).
    """
    jacobian_field = np.zeros((*deformation_field.shape[:3], 3, 3))

    # Calculate partial derivatives using finite differences
    for i in range(3):  # x, y, z components
        jacobian_field[..., i, 0] = sobel(deformation_field[..., i], axis=0, mode='nearest')  # ∂u/∂x, ∂v/∂x, ∂w/∂x
        jacobian_field[..., i, 1] = sobel(deformation_field[..., i], axis=1, mode='nearest')  # ∂u/∂y, ∂v/∂y, ∂w/∂y
        jacobian_field[..., i, 2] = sobel(deformation_field[..., i], axis=2, mode='nearest')  # ∂u/∂z, ∂v/∂z, ∂w/∂z

    # Add identity matrix to account for the fact we’re working with a displacement field
    identity = np.eye(3)
    jacobian_field += identity[None, None, None, :, :]

    return jacobian_field


def func3(fixed_image, dti_image, fwd_transforms):
    """
    Vectorized relocation and reorientation of a 6-component DTI image based on a warp field.

    Parameters:
    - fixed_image: ANTsImage, reference image (e.g., FA or b0)
    - dti_image: ANTsImage, the 6-component DTI image (4D image)
    - transforms: str-pair, paths to the forward transformations

    Returns:
    - ANTsImage, the reoriented 6-component DTI image
    """
    # Step 0: Combine Warps
    combined_warp = combine_warp(fwd_transforms) 

    # Step 1: Split the 6-component DTI image
    dti_components = ants.split_channels(dti_image)
    
    # Step 2: Relocate each component by applying the warp field
    relocated_components = [
        ants.apply_transforms(fixed=fixed_image, moving=component, transformlist=fwd_transforms)
        for component in dti_components
    ]
    
    # Convert relocated components to a 4D array: shape = (X, Y, Z, 6)
    relocated_dti_array = np.stack([comp.numpy() for comp in relocated_components], axis=-1)

    # Step 3: Compute the Jacobian from the warp field as a 4D array
    jacobian_array = compute_jacobian_field(combined_warp.numpy())

    # Step 4: Construct the 3x3 tensor for each voxel using broadcasting
    T = np.zeros((*relocated_dti_array.shape[:3], 3, 3))
    T[..., 0, 0] = relocated_dti_array[..., 0]  # Dxx
    T[..., 0, 1] = relocated_dti_array[..., 1]  # Dxy
    T[..., 1, 0] = relocated_dti_array[..., 1]  # Dxy
    T[..., 0, 2] = relocated_dti_array[..., 2]  # Dxz
    T[..., 2, 0] = relocated_dti_array[..., 2]  # Dxz
    T[..., 1, 1] = relocated_dti_array[..., 3]  # Dyy
    T[..., 1, 2] = relocated_dti_array[..., 4]  # Dyz
    T[..., 2, 1] = relocated_dti_array[..., 4]  # Dyz
    T[..., 2, 2] = relocated_dti_array[..., 5]  # Dzz

    # Step 5: Reorient the tensor using batched matrix multiplication
    # Reoriented tensor T' = J * T * J^T, performed across all voxels simultaneously
    reoriented_T_2 = np.einsum('...ij,...jk,...lk->...il', jacobian_array, T, jacobian_array)

    # First, perform the multiplication J * T for each voxel
    JT = np.matmul(jacobian_array, T)

    # Then, multiply by J^T (transpose of jacobian_field along the last two axes)
    reoriented_T = np.matmul(JT, jacobian_array.transpose(0, 1, 2, 4, 3))

    print("MAE ", np.mean(np.abs(reoriented_T, reoriented_T_2)))

    # Step 6: Extract the reoriented components from the reoriented tensor
    reoriented_dti = np.zeros((*relocated_dti_array.shape[:3], 6))
    reoriented_dti[..., 0] = reoriented_T[..., 0, 0]  # Dxx
    reoriented_dti[..., 1] = reoriented_T[..., 0, 1]  # Dxy
    reoriented_dti[..., 2] = reoriented_T[..., 0, 2]  # Dxz
    reoriented_dti[..., 3] = reoriented_T[..., 1, 1]  # Dyy
    reoriented_dti[..., 4] = reoriented_T[..., 1, 2]  # Dyz
    reoriented_dti[..., 5] = reoriented_T[..., 2, 2]  # Dzz

    print(reoriented_dti.shape)

    # Step 7: Convert the reoriented tensor back into an ANTsImage
    reoriented_dti_image = ants.from_numpy(reoriented_dti, 
                                           origin=fixed_image.origin, 
                                           spacing=fixed_image.spacing, 
                                           direction=fixed_image.direction, has_components=True)

    return reoriented_dti_image

rotated_tensor3 = func3(patient, tensor_image, transform)

MAE  0.0023838751778361986
(240, 240, 155, 6)


In [69]:
def compute_jacobian_field(deformation_field, spacing):
    """
    Efficiently compute Jacobian matrices for entire deformation field.
    
    Parameters:
    -----------
    deformation_field : numpy.ndarray (shape=[x, y, z, 3])
        The deformation field
    spacing : tuple
        Voxel spacing in mm
    
    Returns:
    --------
    numpy.ndarray (shape=[x, y, z, 3, 3])
        Jacobian matrices for each voxel
    """
    # Initialize gradients array
    gradients = np.zeros((*deformation_field.shape[:3], 3, 3))
    
    # Compute gradients using numpy's gradient function
    for i in range(3):
        grad = np.gradient(deformation_field[..., i], 
                         spacing[0], spacing[1], spacing[2])
        for j in range(3):
            gradients[..., i, j] = grad[j]
    
    # Add identity matrix to gradients
    gradients += np.eye(3)
    
    return gradients

def compute_rotation_field(jacobian_field):
    """
    Extract rotation matrices from Jacobian field using vectorized SVD.
    
    Parameters:
    -----------
    jacobian_field : numpy.ndarray (shape=[x, y, z, 3, 3])
        Field of Jacobian matrices
    
    Returns:
    --------
    numpy.ndarray (shape=[x, y, z, 3, 3])
        Field of rotation matrices
    """
    shape = jacobian_field.shape[:-2]
    
    # Reshape for vectorized SVD
    J_flat = jacobian_field.reshape(-1, 3, 3)
    
    # Perform SVD
    U, _, Vh = np.linalg.svd(J_flat)
    
    # Compute rotation matrices
    R = np.einsum('...ij,...jk->...ik', U, Vh)
    
    # Fix improper rotations
    det = np.linalg.det(R)
    fix_idx = det < 0
    U[fix_idx, :, -1] *= -1
    R[fix_idx] = np.einsum('...ij,...jk->...ik', U[fix_idx], Vh[fix_idx])
    
    return R.reshape(*shape, 3, 3)


def func4(fixed_image, dti_image, fwd_transforms):
    """
    Transform DTI data using vectorized operations -> Claude implementation 
    """
    print(dti_image)

    dti_arr = dti_image.numpy()
    deformation_field = combine_warp(fwd_transforms).numpy()

    shape = dti_arr.shape[:3]
    
    # Create coordinate grid
    grid = np.stack(np.meshgrid(
        np.arange(shape[0]),
        np.arange(shape[1]),
        np.arange(shape[2]),
        indexing='ij'
    ), axis=-1).astype(float)
    
    # Apply deformation to coordinates
    new_coords = grid + deformation_field
    
    # Compute Jacobian field
    jacobian_field = compute_jacobian_field(deformation_field, fixed_image.spacing)
    
    # Get rotation field
    rotation_field = compute_rotation_field(jacobian_field)
    
    # Convert 6D vectors to 3x3 matrices
    tensor_matrices = np.zeros((*shape, 3, 3))
    tensor_matrices[..., 0, 0] = dti_arr[..., 0]  # Dxx
    tensor_matrices[..., 0, 1] = tensor_matrices[..., 1, 0] = dti_arr[..., 1]  # Dxy
    tensor_matrices[..., 0, 2] = tensor_matrices[..., 2, 0] = dti_arr[..., 2]  # Dxz
    tensor_matrices[..., 1, 1] = dti_arr[..., 3]  # Dyy
    tensor_matrices[..., 1, 2] = tensor_matrices[..., 2, 1] = dti_arr[..., 4]  # Dyz
    tensor_matrices[..., 2, 2] = dti_arr[..., 5]  # Dzz
    
    # Apply rotation to tensors: R * T * R^T
    # First multiply: T * R^T
    temp = np.einsum('...ij,...jk->...ik', tensor_matrices, np.transpose(rotation_field, (0,1,2,4,3)))
    # Then multiply: R * (T * R^T)
    rotated_tensors = np.einsum('...ij,...jk->...ik', rotation_field, temp)
    
    # Convert back to 6D vectors
    transformed_dti = np.zeros_like(dti_arr)
    transformed_dti[..., 0] = rotated_tensors[..., 0, 0]  # Dxx
    transformed_dti[..., 1] = rotated_tensors[..., 0, 1]  # Dxy
    transformed_dti[..., 2] = rotated_tensors[..., 0, 2]  # Dxz
    transformed_dti[..., 3] = rotated_tensors[..., 1, 1]  # Dyy
    transformed_dti[..., 4] = rotated_tensors[..., 1, 2]  # Dyz
    transformed_dti[..., 5] = rotated_tensors[..., 2, 2]  # Dzz
    
    # Apply spatial transformation using interpolation
    coords = new_coords.reshape(-1, 3)
    valid_mask = np.all((coords >= 0) & (coords < np.array(shape)), axis=1)
    
    result = np.zeros_like(transformed_dti)
    for i in range(6):
        coordinates = np.moveaxis(new_coords, -1, 0)
        result[..., i] = map_coordinates(transformed_dti[..., i], coordinates, order=1, mode='nearest')
    


    return ants.from_numpy(result, (0.0, -239.0, 0.0), fixed_image.spacing, fixed_image.direction, has_components=True)

rotated_tensor4 = func4(patient, tensor_image, transform)

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (239.0, 0.0, 0.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



In [70]:

print(transformed_tensor_img)
print(rotated_tensor1)
#print(rotated_tensor2)
print(rotated_tensor3)
print(rotated_tensor4)

print(patient)

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1.

In [71]:
print("COMPARISON\n===========")

x, y, z = 100, 100, 100

transformed_tensor_arr = transformed_tensor_img.numpy()
rotated_tensor_arr1 = rotated_tensor1.numpy()
#rotated_tensor_arr2 = rotated_tensor2.numpy()
rotated_tensor_arr3 = rotated_tensor4.numpy()
rotated_tensor_arr4 = rotated_tensor4.numpy()

print(transformed_tensor_arr[x, y, z])
print(rotated_tensor_arr1[x, y, z])
print(rotated_tensor_arr2[x, y, z])

print("Comparing my method with the Ants method")
print("MSE: ", np.mean(np.square(rotated_tensor_arr1 - rotated_tensor_arr3)))
print("MAE: ", np.mean(np.abs(rotated_tensor_arr1 - rotated_tensor_arr3)), "\n\n")

print("Comparing my method with the original tensor image")
print("MSE: ", np.mean(np.square(transformed_tensor_arr - rotated_tensor_arr3)))
print("MAE: ", np.mean(np.abs(transformed_tensor_arr - rotated_tensor_arr3)))

print("Comparing ANTS method with the original tensor image")
print("MSE: ", np.mean(np.square(transformed_tensor_arr - rotated_tensor_arr1)))
print("MAE: ", np.mean(np.abs(transformed_tensor_arr - rotated_tensor_arr1)))

COMPARISON
[ 7.8866823e-04  1.6968910e-05  7.9412370e-05  8.7626872e-04
 -3.3947985e-05  9.1161032e-04]
[ 7.8185508e-04  1.1979826e-05  7.6760836e-05  8.7911834e-04
 -3.0642372e-05  9.1557385e-04]
[ 7.8866823e-04  1.6968910e-05  7.9412370e-05  8.7626872e-04
 -3.3947985e-05  9.1161032e-04]
Comparing my method with the Ants method
MSE:  6.2243984e-08
MAE:  7.001484e-05 


Comparing my method with the original tensor image
MSE:  6.2242556e-08
MAE:  7.0020826e-05
Comparing ANTS method with the original tensor image
MSE:  1.0579147e-11
MAE:  7.350872e-07


In [72]:
ants.image_write(rotated_tensor4, "out/transformed_rotated_dti_test.nii.gz")